# Homework 7 Template

Rixin Li & G . Besla

look for "****"

In [14]:
# import necessary modules
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# matplotlib provides powerful functions for plotting figures
import matplotlib.pyplot as plt
# astropy provides unit system and constants for astronomical calculations
import astropy.units as u
import astropy.constants as const
# import Latex module so we can display the results with symbols
from IPython.display import Latex
%matplotlib inline

# **** import CenterOfMass to determine the COM pos/vel of M33
from CenterOfMass import CenterOfMass

# **** import the GalaxyMass to determine the mass of M31 for each component
from GalaxyMass import ComponentMass

ModuleNotFoundError: No module named 'CenterOfMass'

# M33AnalyticOrbit

In [10]:
class M33AnalyticOrbit:
    """ Calculate the analytical orbit of M33 around M31 """
    
    def __init__(self, filename): # **** add inputs
                """ 
    # Initialize the instance of this Class with the following properties:
    # filename : filename for the file in which to store in the integrated orbit """

        ### get the gravitational constant (the value is 4.498502151575286e-06)
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).
        
        ### **** store the output file name
        # compose the filename for output
        #fileout = "AnalyticOrbit_" + galaxy + ".txt"
        fileout = 'AnalyticOrbit_%s.txt'%(filename)   # filename to store output
        
        ### get the current pos/vel of M33 
        # **** create an instance of the  CenterOfMass class for M33 
        COM_M33 = CenterOfMass("M33_000.txt",2) 
        # **** store the position VECTOR of the M33 COM (.value to get rid of units)
        M33_COM_P = M33COM.COM_P(0.1).value
        # **** store the velocity VECTOR of the M33 COM (.value to get rid of units)
        M33_COM_V = M33COM.COM_V(M33_COM_P[0].value,M33_COM_P[1].value,M33_COM_P[2].value)

        
        ### get the current pos/vel of M31 
        # **** create an instance of the  CenterOfMass class for M31 
        COM_M31 = CenterOfMass("M31_000.txt",2) 

        # **** store the position VECTOR of the M31 COM (.value to get rid of units)
        M31_COM_P = M31COM.COM_P(0.1).value
        # **** store the velocity VECTOR of the M31 COM (.value to get rid of units)
        M31_COM_V = M31COM.COM_V(M31_COM_P[0].value,M31_COM_P[1].value,M31_COM_P[2].value)

        
        ### store the DIFFERENCE between the vectors posM33 - posM31
        # **** create two VECTORs self.r0 and self.v0 and have them equal the
        # relative position and velocity VECTORS of M33
        self.r0 = M33_COM_P - M31_COM_P
        self.v0 = M33_COM_V - M33_COM_V
        
        ### get the mass of each component in M31 
        ### disk
        # **** self.rdisk = scale length (no units)
        self.rdisk = 5
        # **** self.Mdisk set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mdisk = ComponentMass("M31_000",2)*1e12
        ### bulge
        # **** self.rbulge = set scale length (no units)
        self.rbulge = 1
        # **** self.Mbulge  set with ComponentMass function. Remember to *1e12 to get the right units Use the right ptype
        M31_bulge = ComponentMass("M31_000.txt",3)*1e12
        # Halo
        # **** self.rhalo = set scale length from HW5 (no units)
        self.rhalo = 62
        # **** self.Mhalo set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        M31_halo = ComponentMass("M31_000.txt",1)*1e12

    
    
    def HenquistAccel(self,M,r_a,x,y,z): # it is easiest if you take as an input the position VECTOR 
        """  
        # Function to compute the Gravitational Acceleration Vector of M31
        # This function wil be used for both Halo and Bulge of M31
        # Inputs:
        #        M is the Total Halo and Bulge Mass 
        #        r_a is the corresponding Scale Length 
        #        x, y, z are the positions of M31 Halo and Bulge
        # Returns:
        #        Acceleration vector from a Hernquist Potential """
        
        ### **** Store the magnitude of the position vector
        r = np.sqrt(x**2 + y**2 + z**2)
        
        ### *** Store the Acceleration
        Hern = -G*M/(r(r_a + r)**2)*r #follow the formula in the HW instructions
        # NOTE: we want an acceleration VECTOR so you need to make sure that in the Hernquist equation you 
        # use  -G*M/(rmag *(ra + rmag)**2) * r --> where the last r is a VECTOR 
        
        return Hern
    
    
    
    def MiyamotoNagaiAccel(self,M,rd,x,y,z):# it is easiest if you take as an input a position VECTOR  r 
               """ 
        Function to compute Disk Acceleration Vector of M31
        # Inputs:
        #        M is the Disk Mass 
        #        rd is the corresponding Scale Length   
        #        x, y, z are the positions of M31 Disk
        # Returns:
        #        Acceleration vector from a Miyamoto-Nagai Profile """
        
        ### Acceleration **** follow the formula in the HW instructions
        zd = self.rdisk/5.0
        R = np.sqrt(x**2 + y**2)
        B = rd + np.sqrt(z**2 + zd**2
        # ZSTUFF are the terms associated with the z direction                 
        ZSTUFF = B/np.sqrt(z**2 + zd**2)
        # Miyamoto-Nagai Profile
        MN_profile = -G*M/(R**2 + B**2)**1.5
        # Acceleration vector                  
        Acc = MN_profile*np.array([1,1,ZSTUFF])
        # Compute the magnitude 
        Acc_mag = np.linalg.norm(Acc)
                
        
        # AGAIN note that we want a VECTOR to be returned  (see Hernquist instructions)
        # this can be tricky given that the z component is different than in the x or y directions. 
        # we can deal with this by multiplying the whole thing by an extra array that accounts for the 
        # differences in the z direction:
        # multiply the whole thing by :   np.array([1,1,ZSTUFF]) 
        # where ZSTUFF are the terms associated with the z direction       
       
        return Acc_mag
        # the np.array allows for a different value for the z component of the acceleration
     
    
    def M31Accel(self,r): # input should include the position vector, r
    """"
    Function commputes the sum of all acceleration
    Inputs: 
           r is the 3D position vector 
    Returns: 
           The 3D total acceleration vector of M31
    """"
        ### Call the previous functions for the halo, bulge and disk
        # **** these functions will take as inputs variable we defined in the initialization of the class like 
        # self.rdisk etc. 
        # Acceleration vector of M31 Bulge
        HernBulge = HernquistAccel(self.Mbulge,self.rbulge,self.x,self.y,self.z)  
        # Acceleration vector of M31 Halo
        HernHalo = HernquistAccel(self.Mhalo,self.rhalo,self.x,self.y,self.z)
        # Acceleration vector of M31 Disk
        MN = MiyamotoNagaiAccel(M,self.rdisk,self.x,self.y,self.z)
        #???????
                         
            # return the SUM of the output of the acceleration functions - this will return a VECTOR 
        return HernBulge + HernHalo + MN
    
    
    
    def LeapFrog(self,dt,x,y,z,vx,vy,vz): # take as input r and v, which are VECTORS. Assume it is ONE vector at a time
                               """ **** ADD COMMENTS """

        r = np.sqrt(x**2 + y**2 + z**2)
        v = np.sqrt(vx**2 + vy**2 + vz**2)
                         
        # predict the position at the next half timestep
        rhalf = r + v*dt/2
        
        # compute the velocity at the next timestep
        #  calling M31 Acceleration function
        Acc_M31 = self.M31Accel(r)
        vnew = v + Acc_M31*dt
        
        # compute the position at the next timestep
        rnew = r + (v+vnew)*dt/2
        
        return rnew, vnew # **** return the new position and velcoity vectors
    
    
    
    def OrbitIntegration(self, t0, dt, tmax):
         """ **** ADD COMMENTS """

        # initialize the time to the input starting time
        t = t0
        
        # initialize an empty array of size :  rows int(tmax/dt)+2  , columns 7
        orbit = np.zeros((size(int(tmax/dt)+2)),7)
        
        # initialize the first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        # this above is equivalent to 
        # orbit[0] = t0, self.r0[0], self.r0[1], self.r0[2], self.v0[0], self.v0[1], self.v0[2]
        
        
        # initialize a counter for the orbit.  
        i = 1 # since we already set the 0th values, we start the counter at 1
        
        # start the integration (advancing in time steps and computing LeapFrog at each step)
        while (t<tmax):  # as long as t has not exceeded the maximal time 
            
            # **** advance the time by one timestep, dt
            t = t + dt
           
            # **** store the new time in the first element of the ith row
            orbit[i,0] = orbit[i,0].append(t)
            
            # ***** advance the position and velocity using the LeapFrog scheme
            rnew, vnew = LeapFrog(dt,x,y,z,vx,vy,vz)
            # remember that LeapFrog returns a position vector and a velocity vector  
            # as an example, if a function returns three vectors you would call the function and store 
            # the variable like:     a,b,c = function(input)
            
         
    
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            orbit[i,1] = 
            # TIP:  if you want columns 5-7 of the Nth row of an array called A, you would write : 
            # A[n, 5:8] 
            # where the syntax is row n, start at column 5 and end BEFORE column 8
            
            
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            
            
            # **** update counter i , where i is keeping track of the number of rows (i.e. the number of time steps)
        
        
        
        # write the data to a file
        np.savetxt(self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
        
        # there is no return function
        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 95)